# Data_Generator

In [428]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import pennylane as qml
import numpy as np
from numpy import pi
simulator = Aer.get_backend('qasm_simulator')

from scipy.linalg import expm, sinm, cosm

# Ansatz

In [907]:
def Operational_circuit(params):
    qc = QuantumCircuit(1)
    
    #qc.h(range(2))
    qc.u3(params[0],params[1],params[2],0)
    #qc.rx(pi/2,0)
    
    return qc
    

In [908]:
Operational_circuit_try = Operational_circuit([0,1,2])

In [909]:
Operational_circuit_try.draw()

┌───────────┐
q_0: ┤ U3(0,1,2) ├
     └───────────┘

# Measurments for different components of H

# 1. Z

In [910]:
def measure_z_circuit(given_circuit):
    z_meas = given_circuit.copy()
    z_meas.measure_all()
    return z_meas

z_meas = measure_z_circuit(Operational_circuit_try)
z_meas.draw()

┌───────────┐ ░ ┌─┐
   q_0: ┤ U3(0,1,2) ├─░─┤M├
        └───────────┘ ░ └╥┘
meas_0: ═════════════════╩═

In [911]:
def measure_z(given_circuit, num_shots = 10000):

    z_meas = measure_z_circuit(given_circuit)
    
    result = execute(z_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(z_meas)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    total_counts = counts['0'] + counts['1']
    z = counts['0'] - counts['1']
    z = z / total_counts
    
    return z

In [912]:
z = measure_z(Operational_circuit_try)
print("<Z> =", str(z))

<Z> = 1.0


# 2. X

In [913]:
def measure_x_circuit(given_circuit):
    x_meas = given_circuit.copy()
    x_meas.h(0)
    x_meas.measure_all()

    return x_meas

In [914]:
x_meas = measure_x_circuit(Operational_circuit_try)
x_meas.draw()

┌───────────┐┌───┐ ░ ┌─┐
   q_0: ┤ U3(0,1,2) ├┤ H ├─░─┤M├
        └───────────┘└───┘ ░ └╥┘
meas_0: ══════════════════════╩═

In [915]:
def measure_x(given_circuit, num_shots = 10000):
    
    x_meas = measure_x_circuit(given_circuit)
    
    result = execute(x_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(x_meas)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    total_counts = counts['0'] + counts['1'] 
    x = counts['0'] - counts['1']
    x = x / total_counts
    
    return x

In [916]:
x = measure_x(Operational_circuit_try)
print("<X> =", str(x))

<X> = 0.0022


# 3. Y

In [917]:
def measure_y_circuit(given_circuit):
    y_meas = given_circuit.copy()
    y_meas.rz(-pi/2,0)
    y_meas.h(0)

    y_meas.measure_all()

    return y_meas

In [918]:
y_meas = measure_y_circuit(Operational_circuit_try)
y_meas.draw()

┌───────────┐┌───────────┐┌───┐ ░ ┌─┐
   q_0: ┤ U3(0,1,2) ├┤ RZ(-pi/2) ├┤ H ├─░─┤M├
        └───────────┘└───────────┘└───┘ ░ └╥┘
meas_0: ═══════════════════════════════════╩═

In [948]:
def measure_y(given_circuit, num_shots = 10000):
    
    y_meas = measure_y_circuit(given_circuit)
    
    result = execute(y_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(y_meas)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    total_counts = counts['0'] + counts['1'] 
    y = counts['0']  - counts['1']
    y = y / total_counts
    
    return y

In [949]:
y = measure_y(Operational_circuit_try)
print("<y> =", str(y))

<y> = -0.0016


# Matrix and Decomposition

In [950]:
M= np.matrix('0 -1j  ;1j 0 ')
M=1*M

In [951]:
coeffs, obs_list = qml.utils.decompose_hamiltonian(M)
H = qml.Hamiltonian(coeffs, obs_list)

In [952]:
print(H)

(1.0) [Y0]


In [953]:
def coef(H):
    coeff = [0,0,0,0]
    h = str(H)
    for i in range (len(h)):
        p = ""
        if h[i] =="(":
            j = i+1 
            while h[j]!=")" :
                p=p+h[j]
                j+=1
            
            if h[j+3] == "I":
                coeff[0] = float(p)
                
            if h[j+3] == "X":
                coeff[1] = float(p)
                
            if h[j+3] == "Y":
                coeff[2] = float(p)
                
            if h[j+3] == "Z":
                coeff[3] = float(p)
    
    return(coeff)
            
            

# Cost_Function

In [954]:
coef(H) 

[0, 0, 1.0, 0]

In [955]:
def eigen_value(params):
    num_shots = 100000
    z = measure_z(Operational_circuit(params), num_shots = num_shots)
    x = measure_x(Operational_circuit(params), num_shots = num_shots)
    y = measure_y(Operational_circuit(params), num_shots = num_shots)
    coeff =coef(H) 
    ev = coeff[0]+coeff[1]*x+coeff[2]*y+coeff[3]*z
    
    return ev

In [958]:
params = [pi/2,-pi/2,pi/2]

In [959]:
ev = eigen_value(params)    
print("The Eigen_Value of the trial state is", str(ev))

The Eigen_Value of the trial state is -1.0


# Finding Lowest Eignevalue

In [960]:
from qiskit.aqua.components.optimizers import COBYLA,SLSQP,SPSA

optimizer = COBYLA(50,0.001)

In [961]:
ret = optimizer.optimize(num_vars=3, objective_function=eigen_value, initial_point=params)

In [962]:
eigen_value(ret[0])

-1.0

In [963]:
ret

(array([ 1.57079633, -1.57079633,  1.57079633]), -1.0, 28)

# Eigen State

In [964]:
backend_state = Aer.get_backend('statevector_simulator')

In [965]:

result = execute(Operational_circuit(ret[0]),backend_state).result()
out_state = result.get_statevector()


In [966]:
print(out_state)

[7.07106781e-01+0.j         4.32978028e-17-0.70710678j]


# Check

In [967]:
qc = QuantumCircuit(1)
qc.initialize(out_state,0)

In [968]:
def eigen_value_state(qc):
    num_shots = 100000
    z = measure_z(qc, num_shots = num_shots)
    x = measure_x(qc, num_shots = num_shots)
    y = measure_y(qc, num_shots = num_shots)
    coeff =coef(H) 
    ev = coeff[0]+coeff[1]*x+coeff[2]*y+coeff[3]*z
    
    return ev

In [969]:
 eigen_value_state(qc)

-1.0

# Exponential Matrix 

In [970]:
from scipy.linalg import expm, sinm, cosm

In [971]:
def mult(X,Y):
    Z = [0,0,0,0]
    Z[0] = round(expm(X)[0][0]*expm(Y)[0][0]+expm(X)[0][1]*expm(Y)[1][0])
    Z[1] = round(expm(X)[0][0]*expm(Y)[0][1]+expm(X)[0][1]*expm(Y)[1][1])
    Z[2] = round(expm(X)[1][0]*expm(Y)[0][0]+expm(X)[1][1]*expm(Y)[1][0])
    Z[3] = round(expm(X)[1][0]*expm(Y)[0][1]+expm(X)[1][1]*expm(Y)[1][1])
    return(Z)

In [972]:
had = [(0.5)**2, (0.5)**2]
initial_h = np.dot(expm(1j*M),had)
initial = np.dot(expm(1j*M),[1,0])

In [973]:
print("Matrix:-", M)
print()
print("e^iH:-",expm(1j*M) )
print()
print("Initial_H:-", initial_h)
print()
print("Initial_Down:-", initial)
print()
print("Output:-",out_state)

Matrix:- [[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]

e^iH:- [[ 0.54030231+0.j  0.84147098+0.j]
 [-0.84147098+0.j  0.54030231+0.j]]

Initial_H:- [ 0.34544332+0.j -0.07529217+0.j]

Initial_Down:- [ 0.54030231+0.j -0.84147098+0.j]

Output:- [7.07106781e-01+0.j         4.32978028e-17-0.70710678j]
